In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy import stats
from scipy.stats import linregress
from config import weather_api_key
#from config import g_key
from citipy import citipy
import csv as atlantic_working

In [2]:
# Import API key
from config import weather_api_key

In [3]:
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

In [4]:
awdf = pd.read_csv('atlantic macro.csv')
awdf['Latitude'].head()
awdf['Longitude'].head()

0    94.8W
1    95.4W
2    96.0W
3    96.5W
4    96.8W
Name: Longitude, dtype: object

In [5]:
#set "city center" for Houston
Houston_lat = 29.756097853207656
Houston_lng = -95.3669907972107


#calculate greater Houston area by expanding lat and lng
lat = np.random.uniform(Houston_lat-1, Houston_lat+2, size=6500)
lng = np.random.uniform(Houston_lng-5, Houston_lng+2, size=6500)
city = []

#find nearest city using lat and lng
for x in range(len(lat)):
    city.append(citipy.nearest_city(lat[x], lng[x]).city_name)

In [ ]:
#turn cities into df
df_city = pd.DataFrame({
    "City":city,
    "lat":lat,
    "lng":lng,
})

In [6]:
awdf

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum PreLow Intensity Subtropical Cycloneure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,AL011851,UNNAMED,18510625,0,,Hurricane,28.0N,94.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,AL011851,UNNAMED,18510625,600,,Hurricane,28.0N,95.4W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,18510625,1200,,Hurricane,28.0N,96.0W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,18510625,1800,,Hurricane,28.1N,96.5W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,18510625,2100,Landfall,Hurricane,28.2N,96.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49100,AL122015,KATE,20151112,1200,,Extratropical Cyclone,41.3N,50.4W,55,981,...,180,120,120,120,60,0,0,0,0,0
49101,AL122015,KATE,20151112,1800,,Extratropical Cyclone,41.9N,49.9W,55,983,...,180,120,120,120,60,0,0,0,0,0
49102,AL122015,KATE,20151113,0,,Extratropical Cyclone,41.5N,49.2W,50,985,...,200,220,120,120,60,0,0,0,0,0
49103,AL122015,KATE,20151113,600,,Extratropical Cyclone,40.8N,47.5W,45,985,...,180,220,0,0,0,0,0,0,0,0


In [7]:
lat=awdf["Latitude"].to_string()
lat = awdf.loc[:, "Latitude"]
lat = lat.str.replace('N', "")
awdf.loc[:,'Latitude'] = lat


lng=awdf["Longitude"].to_string()
lng = awdf.loc[:, "Longitude"]
lng = lng.str.replace('W', "")
awdf.loc[:,'Longitude'] = lng.astype('float') * -1

#print(lng)


ValueError: could not convert string to float: '5.0E'

In [ ]:
awdf